In [137]:

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import math

# Start notebook mode
init_notebook_mode(connected=True)

import pandas as pd # we gebruiken pandas om de CSV data te laden en te visualiseren


times = pd.read_csv("timesData.csv")
shanghai = pd.read_csv("shanghaiData.csv")
cwur = pd.read_csv("cwurData.csv")

# print(shanghai.loc[shanghai['university_name'] == 'Harvard University']['world_rank'])

def conv_ranking(x):
    spl = x.split('-')
    if len(spl) == 2:
        return (float(spl[0]) + float(spl[1])) / 2
    s = (x if x[0] != '=' else x[1:])
    f = float(s)
    return f

# countries = pd.read_csv("school_and_country_table.csv")

# df = times.set_index('university_name').join(countries.set_index('school_name'))
# print(df)

times['world_rank'] = times['world_rank'].map(conv_ranking)
country_stats = times[['country', 'world_rank', 'year']].groupby(['year', 'country']).mean()
print(list(x for x in country_stats.index.levels[0]))
# print(country_stats.loc[2011])

[2011, 2012, 2013, 2014, 2015, 2016]


In [157]:

def choro(raw, year):
    return go.Choropleth(
        colorscale = [[0, 'rgb(50,50,255)'], [1, 'rgb(255,50,50)']],
        autocolorscale = False,
        locations = raw.index, #countries['country'],
        z = raw['world_rank'].astype(float),
        locationmode = 'country names',
#         text = raw.index,
        marker = go.choropleth.Marker(
            line = go.choropleth.marker.Line(
                color = 'rgb(0,0,0)',
                width = .25
            )),
        colorbar = go.choropleth.ColorBar(
            title = "Average Ranking %s (high is bad)" % str(year)),
        customdata = [year]
    )

data = [choro(country_stats.loc[year], year) for year in country_stats.index.levels[0]]

steps = []
for i, d in enumerate(data):
    step = dict(method='restyle',
                args=['visible', [False] * (len(data))],
                label='Year {}'.format(d.customdata[0]))
    step['args'][1][i] = True
    steps.append(step)
    
sliders = [dict(active=(len(data)-1),
                pad={"t": 1},
                steps=steps)]  

layout = go.Layout(
    title = go.layout.Title(
        text = 'Average ranking of universities by country (White means no data)'
    ),
    geo = go.layout.Geo(
        scope = 'world',
        projection = go.layout.geo.Projection(type = 'equirectangular'),
        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)'),
    sliders = sliders
)

fig = go.Figure(data = data, layout = layout)
iplot(fig)



rankings = {} # rankings[universiteit] = [times, shanghai, cwur]

def rename(university_name):
    univ = university_name.split(',')[0].strip()
    univ = univ.split('(')[0].strip()
        
    return univ

print('times')
for world_rank, year, university_name in zip(times['world_rank'], times['year'], times['university_name']):
    if year not in rankings:
        rankings[year] = {}
    rankings[year][rename(university_name)] = world_rank
    
print('shanghai')
for world_rank, year, university_name in zip(shanghai['world_rank'], shanghai['year'], shanghai['university_name']):
    if year not in rankings :
        continue
        
    if univ
    rankings[year][] += world_rank
    
print('cwur')
for world_rank, year, university_name in zip(cwur['world_rank'], cwur['year'], cwur['institution']):
    if year not in rankings:
        continue
    rankings[year][rename(university_name)] += world_rank
    
for year in rankings:
    print('year')
    for univ in rankings[year]:
        print(univ + ' ' + rankings[year][univ])

# print(times['world_rank'], shanghai['world_rank'], cwur['world_rank'])

# mashed = pd.DataFrame()
